In [13]:
import pandas as pd
import requests

soccer_url = "https://raw.githubusercontent.com/BriChavez/FIFAWorldCup/master/squads.csv"
df = pd.read_csv(soccer_url)

df = df.set_index('Player')




In [15]:
df = df.fillna('n/a')
df = df.fillna(0)

df.columns = ['Jersey_Num', 'Position', 'DOB_Age',
              'Caps', 'Club', 'Country', 'ClubCountry', 'Cup_Year']


In [9]:
df.to_csv('data/world_cup.csv')

In [16]:
# Imports the Google Cloud client library
import os
from google.cloud import storage

# Instantiates a client

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/fossa/data/dont_worry_about_it/dbt-user-creds.json"
client = storage.Client()


# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/world_cup/
bucket = client.get_bucket('world_cup')
# upload the csv to the bucket
blob = bucket.blob('world_cup.csv')
blob.upload_from_filename('data/world_cup.csv')


Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/world_cup?projection=noAcl&prettyPrint=false: dbt-user@dbtweek.iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket.